In [ ]:
import os
from glob import glob

from IPython.display import Markdown, display

import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate

import pydicom

import pymedphys.level1.dcmutilities as dcmutilities

In [ ]:
dicom_data_directory = '../../data/collapsedcone/dicom'
keys = ['10x10', '40x40']

In [ ]:
fluence_dicom_files = {
    key: glob(os.path.join(
        dicom_data_directory, 'FLUENCE*{}*.dcm').format(key))[0]
    for key in keys
}
fluence_dicom_files

In [ ]:
reference_dose_files = {
    key: glob(os.path.join(
        dicom_data_directory, 'QABOX*{}*.dcm').format(key))[0]
    for key in keys
}
reference_dose_files

In [ ]:
dcm_fluence = {
    key: pydicom.read_file(fluence_dicom_files[key], force=True)
    for key in keys
}

dose_fluence = {
    key: dcmutilities.load_dose_from_dicom(dcm_fluence[key])
    for key in keys
}

coords_fluence = {
    key: dcmutilities.load_xyz_from_dicom(dcm_fluence[key])
    for key in keys
}

In [ ]:
approx_fluence = {
    key: np.swapaxes(np.mean(dose_fluence[key][1:3,:,:], axis=0), 0, 1)
    for key in dose_fluence
}

In [ ]:
display(Markdown('## Approximate Agility Head Fluence'))

for key in keys:
    plt.figure()
    plt.contourf(coords_fluence[key][0], coords_fluence[key][2], approx_fluence[key], 100)
    plt.colorbar()
    plt.axis('equal')
    
    plt.title(key)
    plt.xlabel('x (mm)')
    plt.ylabel('z (mm)')

In [ ]:
def create_fractional_photon_spectrum(mu, sigma):
    
    def fractional_photon_spectrum(energy):
        return (
            1 / (np.sqrt(2*np.pi) * sigma * energy) *
            np.exp(-((-np.log(energy) - mu)**2/(2*sigma**2)))
        )
        
    return fractional_photon_spectrum


fractional_photon_spectrum = create_fractional_photon_spectrum(2, 6)

E = np.linspace(1,6,100)
W = fractional_photon_spectrum(E)

plt.plot(E, W)

In [ ]:
# Water https://physics.nist.gov/PhysRefData/XrayMassCoef/ComTab/water.html

NIST_water_attenuation_table = np.array([
    [1.00000E-03, 4.078E+03, 4.065E+03],
    [1.50000E-03, 1.376E+03, 1.372E+03],
    [2.00000E-03, 6.173E+02, 6.152E+02],
    [3.00000E-03, 1.929E+02, 1.917E+02],
    [4.00000E-03, 8.278E+01, 8.191E+01],
    [5.00000E-03, 4.258E+01, 4.188E+01],
    [6.00000E-03, 2.464E+01, 2.405E+01],
    [8.00000E-03, 1.037E+01, 9.915E+00],
    [1.00000E-02, 5.329E+00, 4.944E+00],
    [1.50000E-02, 1.673E+00, 1.374E+00],
    [2.00000E-02, 8.096E-01, 5.503E-01],
    [3.00000E-02, 3.756E-01, 1.557E-01],
    [4.00000E-02, 2.683E-01, 6.947E-02],
    [5.00000E-02, 2.269E-01, 4.223E-02],
    [6.00000E-02, 2.059E-01, 3.190E-02],
    [8.00000E-02, 1.837E-01, 2.597E-02],
    [1.00000E-01, 1.707E-01, 2.546E-02],
    [1.50000E-01, 1.505E-01, 2.764E-02],
    [2.00000E-01, 1.370E-01, 2.967E-02],
    [3.00000E-01, 1.186E-01, 3.192E-02],
    [4.00000E-01, 1.061E-01, 3.279E-02],
    [5.00000E-01, 9.687E-02, 3.299E-02],
    [6.00000E-01, 8.956E-02, 3.284E-02],
    [8.00000E-01, 7.865E-02, 3.206E-02],
    [1.00000E+00, 7.072E-02, 3.103E-02],
    [1.25000E+00, 6.323E-02, 2.965E-02],
    [1.50000E+00, 5.754E-02, 2.833E-02],
    [2.00000E+00, 4.942E-02, 2.608E-02],
    [3.00000E+00, 3.969E-02, 2.281E-02],
    [4.00000E+00, 3.403E-02, 2.066E-02],
    [5.00000E+00, 3.031E-02, 1.915E-02],
    [6.00000E+00, 2.770E-02, 1.806E-02],
    [8.00000E+00, 2.429E-02, 1.658E-02],
    [1.00000E+01, 2.219E-02, 1.566E-02],
    [1.50000E+01, 1.941E-02, 1.441E-02],
    [2.00000E+01, 1.813E-02, 1.382E-02]
]).T

plt.loglog(NIST_water_attenuation_table[0], NIST_water_attenuation_table[1])
plt.xlabel('Photon Energy (MeV)')
plt.ylabel(r'$\mu / \rho$ (cm$^2$/g)')

In [ ]:
# Tungsten https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z74.html

NIST_tungsten_attenuation_table = np.array([
    [1.00000E-03, 3.683E+03, 3.671E+03],
    [1.50000E-03, 1.643E+03, 1.632E+03],
    [1.80920E-03, 1.108E+03, 1.097E+03],
    [1.80920E-03, 1.327E+03, 1.311E+03],
    [1.84014E-03, 1.911E+03, 1.883E+03],
    [1.87160E-03, 2.901E+03, 2.853E+03],
    [1.87160E-03, 3.170E+03, 3.116E+03],
    [2.00000E-03, 3.922E+03, 3.853E+03],
    [2.28100E-03, 2.828E+03, 2.781E+03],
    [2.28100E-03, 3.279E+03, 3.226E+03],
    [2.42350E-03, 2.833E+03, 2.786E+03],
    [2.57490E-03, 2.445E+03, 2.407E+03],
    [2.57490E-03, 2.599E+03, 2.558E+03],
    [2.69447E-03, 2.339E+03, 2.301E+03],
    [2.81960E-03, 2.104E+03, 2.071E+03],
    [2.81960E-03, 2.194E+03, 2.160E+03],
    [3.00000E-03, 1.902E+03, 1.873E+03],
    [4.00000E-03, 9.564E+02, 9.405E+02],
    [5.00000E-03, 5.534E+02, 5.423E+02],
    [6.00000E-03, 3.514E+02, 3.428E+02],
    [8.00000E-03, 1.705E+02, 1.643E+02],
    [1.00000E-02, 9.691E+01, 9.204E+01],
    [1.02068E-02, 9.201E+01, 8.724E+01],
    [1.02068E-02, 2.334E+02, 1.966E+02],
    [1.08548E-02, 1.983E+02, 1.684E+02],
    [1.15440E-02, 1.689E+02, 1.444E+02],
    [1.15440E-02, 2.312E+02, 1.889E+02],
    [1.18186E-02, 2.268E+02, 1.797E+02],
    [1.20998E-02, 2.065E+02, 1.699E+02],
    [1.20998E-02, 2.382E+02, 1.948E+02],
    [1.50000E-02, 1.389E+02, 1.172E+02],
    [2.00000E-02, 6.573E+01, 5.697E+01],
    [3.00000E-02, 2.273E+01, 1.991E+01],
    [4.00000E-02, 1.067E+01, 9.240E+00],
    [5.00000E-02, 5.949E+00, 5.050E+00],
    [6.00000E-02, 3.713E+00, 3.070E+00],
    [6.95250E-02, 2.552E+00, 2.049E+00],
    [6.95250E-02, 1.123E+01, 3.212E+00],
    [8.00000E-02, 7.810E+00, 2.879E+00],
    [1.00000E-01, 4.438E+00, 2.100E+00],
    [1.50000E-01, 1.581E+00, 9.378E-01],
    [2.00000E-01, 7.844E-01, 4.913E-01],
    [3.00000E-01, 3.238E-01, 1.973E-01],
    [4.00000E-01, 1.925E-01, 1.100E-01],
    [5.00000E-01, 1.378E-01, 7.440E-02],
    [6.00000E-01, 1.093E-01, 5.673E-02],
    [8.00000E-01, 8.066E-02, 4.028E-02],
    [1.00000E+00, 6.618E-02, 3.276E-02],
    [1.25000E+00, 5.577E-02, 2.761E-02],
    [1.50000E+00, 5.000E-02, 2.484E-02],
    [2.00000E+00, 4.433E-02, 2.256E-02],
    [3.00000E+00, 4.075E-02, 2.236E-02],
    [4.00000E+00, 4.038E-02, 2.363E-02],
    [5.00000E+00, 4.103E-02, 2.510E-02],
    [6.00000E+00, 4.210E-02, 2.649E-02],
    [8.00000E+00, 4.472E-02, 2.886E-02],
    [1.00000E+01, 4.747E-02, 3.072E-02],
    [1.50000E+01, 5.384E-02, 3.360E-02],
    [2.00000E+01, 5.893E-02, 3.475E-02]
]).T

plt.loglog(NIST_tungsten_attenuation_table[0], NIST_tungsten_attenuation_table[1])
plt.xlabel('Photon Energy (MeV)')
plt.ylabel(r'$\mu / \rho$ (cm$^2$/g)')

In [ ]:
# Iron https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z26.html

NIST_iron_attenuation_table = np.array([
    [1.00000E-03, 9.085E+03, 9.052E+03],
    [1.50000E-03, 3.399E+03, 3.388E+03],
    [2.00000E-03, 1.626E+03, 1.620E+03],
    [3.00000E-03, 5.576E+02, 5.535E+02],
    [4.00000E-03, 2.567E+02, 2.536E+02],
    [5.00000E-03, 1.398E+02, 1.372E+02],
    [6.00000E-03, 8.484E+01, 8.265E+01],
    [7.11200E-03, 5.319E+01, 5.133E+01],
    [7.112001E-03, 4.076E+02, 2.978E+02],  # Adjusted for interpolation purposes
    [8.00000E-03, 3.056E+02, 2.316E+02],
    [1.00000E-02, 1.706E+02, 1.369E+02],
    [1.50000E-02, 5.708E+01, 4.896E+01],
    [2.00000E-02, 2.568E+01, 2.260E+01],
    [3.00000E-02, 8.176E+00, 7.251E+00],
    [4.00000E-02, 3.629E+00, 3.155E+00],
    [5.00000E-02, 1.958E+00, 1.638E+00],
    [6.00000E-02, 1.205E+00, 9.555E-01],
    [8.00000E-02, 5.952E-01, 4.104E-01],
    [1.00000E-01, 3.717E-01, 2.177E-01],
    [1.50000E-01, 1.964E-01, 7.961E-02],
    [2.00000E-01, 1.460E-01, 4.825E-02],
    [3.00000E-01, 1.099E-01, 3.361E-02],
    [4.00000E-01, 9.400E-02, 3.039E-02],
    [5.00000E-01, 8.414E-02, 2.914E-02],
    [6.00000E-01, 7.704E-02, 2.836E-02],
    [8.00000E-01, 6.699E-02, 2.714E-02],
    [1.00000E+00, 5.995E-02, 2.603E-02],
    [1.25000E+00, 5.350E-02, 2.472E-02],
    [1.50000E+00, 4.883E-02, 2.360E-02],
    [2.00000E+00, 4.265E-02, 2.199E-02],
    [3.00000E+00, 3.621E-02, 2.042E-02],
    [4.00000E+00, 3.312E-02, 1.990E-02],
    [5.00000E+00, 3.146E-02, 1.983E-02],
    [6.00000E+00, 3.057E-02, 1.997E-02],
    [8.00000E+00, 2.991E-02, 2.050E-02],
    [1.00000E+01, 2.994E-02, 2.108E-02],
    [1.50000E+01, 3.092E-02, 2.221E-02],
    [2.00000E+01, 3.224E-02, 2.292E-0]
]).T

plt.loglog(NIST_iron_attenuation_table[0], NIST_iron_attenuation_table[1], 'o-')
plt.xlabel('Photon Energy (MeV)')
plt.ylabel(r'$\mu / \rho$ (cm$^2$/g)')

# https://physics.nist.gov/PhysRefData/XrayMassCoef/tab1.html
iron_density = 7.874  # g/cm^3

# https://pdfs.semanticscholar.org/9726/2efd27488187b3351516a247e57dd79cf0aa.pdf
approximate_6MV_flatenning_filter_centre_thickness = 2.5 # cm of steel

thickness = approximate_6MV_flatenning_filter_centre_thickness * iron_density  # g / cm^2
thickness

In [ ]:
plt.plot(np.log10(NIST_iron_attenuation_table[0]), np.log10(NIST_iron_attenuation_table[1]), 'o-')

In [ ]:
steel_interp = scipy.interpolate.interp1d(np.log(NIST_iron_attenuation_table[0]), np.log(NIST_iron_attenuation_table[1]))

def steel_attenuation(energy):
    return np.exp(steel_interp(np.log(energy)))

E = np.exp(np.linspace(np.log(0.001), np.log(20), 200))
attenuation = steel_attenuation(E)

plt.loglog(NIST_iron_attenuation_table[0], NIST_iron_attenuation_table[1])
plt.loglog(E, attenuation, '-')

In [ ]:
# A quick and dirty approximation of the energy spectrum

def create_quick_and_dirty_spectrum(mu, sigma, linear_start, cutoff):

    fractional_photon_spectrum = create_fractional_photon_spectrum(mu, sigma)

    def initial_stage_spectrum(energy):
        initial_weight = fractional_photon_spectrum(energy)
        fraction_not_attenuated = np.exp(-steel_attenuation(energy) * thickness)

        return fraction_not_attenuated * initial_weight


    initial_part_of_linear = initial_stage_spectrum(linear_start)
    def linear_portion(energy):
        return (
            (1 - (energy - linear_start) / (cutoff - linear_start)) * 
            initial_part_of_linear)

    
    def get_rough_spectrum_fraction(energy):   
#         weight_of_6MeV

        final_weight = initial_stage_spectrum(energy)
        final_weight[energy>=cutoff] = 0
        in_linear_section = np.logical_and(
            energy>=linear_start, energy<cutoff)
        final_weight[in_linear_section] = linear_portion(
            energy[in_linear_section])

        return final_weight


    E = np.linspace(0.001, cutoff + 0.5, 1000)
    W = fractional_photon_spectrum(E)
    rough_spectrum = get_rough_spectrum_fraction(E)
    normalisation = np.trapz(rough_spectrum, E)

    def normalised_rough_spectrum_fraction(energy):
        return get_rough_spectrum_fraction(energy) / normalisation
    
    return normalised_rough_spectrum_fraction


normalised_rough_spectrum_fraction = create_quick_and_dirty_spectrum(
    0.2, 1.3, 5.7, 6.5)


E = np.linspace(0.001, 7, 1000)
normalised_rough_spectrum = normalised_rough_spectrum_fraction(E)
plt.plot(E, normalised_rough_spectrum)

plt.title('Quick and dirty 6MV spectrum')
plt.xlabel('Energy (MeV)')
plt.ylabel('Normalised fluence fraction')

In [ ]:
water_interp = scipy.interpolate.interp1d(
    np.log(NIST_water_attenuation_table[0]), 
    np.log(NIST_water_attenuation_table[1]))

def water_attenuation(energy):
    return np.exp(water_interp(np.log(energy)))

In [ ]:
def get_mono_terma(energy, water_equivalent_depth):
    return (
        normalised_rough_spectrum_fraction(energy) * 
        # Need to include off axis beam softening in here:
        np.exp(-water_attenuation(energy) * water_equivalent_depth) *
        energy * water_attenuation(energy)
    )


def get_spectrum_terma(water_equivalent_depth):
    energy_steps = np.linspace(0.001, 7, 1000)
    return np.sum(get_mono_terma(
        np.expand_dims(energy_steps, axis=0), 
        np.expand_dims(water_equivalent_depth, axis=1)
    ), axis=1)


depth = np.linspace(0, 30, 200)
terma = get_spectrum_terma(depth)

plt.plot(depth, terma)
plt.ylim([0, None])